In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from sklearn.metrics import confusion_matrix
import pandas as pd
from tensorflow.keras import optimizers
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras import regularizers
import os

In [2]:
import os
import pandas as pd
import librosa
import glob 
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
import sys
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import json

In [6]:
mylist= os.listdir('Data/')

In [7]:
type(mylist)

list

In [8]:
print(mylist[28])

fluent123.wav


In [9]:
print(mylist[28][:-2])

fluent123.w


In [10]:
data, sampling_rate = librosa.load('Data/fluent123.wav')

In [11]:
% pylab inline
import os
import pandas as pd
import librosa
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

UsageError: Line magic function `%` not found.


In [12]:
accent_list=[]
for item in mylist:
    if item[:1]=='f':
        accent_list.append('fluent in english')
    elif item[:1]=='m':
        accent_list.append('Not fluent')

In [13]:
labels = pd.DataFrame(accent_list)

In [ ]:
labels[:30]

In [15]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
        X, sample_rate = librosa.load('Data/'+y, res_type='kaiser_fast',duration=20,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1

In [14]:
df[:5]

,feature
0,"[-19.796509, -19.586632, -19.280352, -19.63063..."
1,"[-14.128404, -15.308195, -15.893202, -15.57884..."
2,"[-25.040691, -24.65103, -23.398838, -22.277956..."
3,"[-41.38101, -40.559868, -38.219265, -37.66, -3..."
4,"[-21.894108, -17.757114, -9.809693, -0.6165192..."


In [16]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [17]:
newdf = pd.concat([df3,labels], axis=1)

In [18]:
rnewdf = newdf.rename(index=str, columns={"0": "label"})

In [19]:
from sklearn.utils import shuffle
rnewdf = shuffle(newdf)
rnewdf[:10]

,0,1,2,3,4,5,6,7,8,9,...,1714,1715,1716,1717,1718,1719,1720,1721,1722,0
152,-12.622986,-12.015111,-13.711258,-11.595865,-11.906852,-12.504929,-12.402363,-12.156275,-12.094319,-10.411689,...,-8.385331,-7.260604,-8.378610,-10.359318,-10.141033,-11.577971,-12.526499,-9.637812,-10.444426,Not fluent
114,-17.714375,-17.520164,-18.430801,-17.580900,-17.657869,-20.414986,-17.442244,-17.214600,-18.556419,-17.531473,...,-16.820585,-19.227779,-20.201162,-19.135582,-18.606953,-18.475815,-16.387560,-16.293743,-18.608173,fluent in english
219,-19.263655,-19.515141,-22.541168,-22.434862,-19.386148,-19.030169,-19.379967,-21.770103,-21.123615,-21.774092,...,-19.291502,-16.871758,-18.566780,-19.095432,-20.017788,-18.350462,-16.942574,-20.177202,-22.401665,Not fluent
132,-21.694620,-21.170483,-22.050428,-22.555704,-23.271702,-22.133074,-20.673618,-19.842907,-20.231371,-20.514259,...,-20.211720,-19.982891,-20.086277,-15.612016,-12.695881,-13.183401,-13.492525,-11.489697,-8.247295,fluent in english
126,-35.964302,-33.272213,-27.484310,-11.189123,-4.737389,-2.142073,-5.232427,-8.291703,-12.154572,-11.074092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fluent in english
141,-1.468907,-1.757589,-1.735902,-3.300912,-6.427239,-7.478268,-9.062500,-11.841684,-16.054874,-17.925184,...,-31.519814,-36.490925,-36.315224,-34.470577,-32.856243,-33.214127,-33.481075,-33.452305,-32.323696,fluent in english
125,-21.488020,-21.257708,-20.215384,-17.234226,-18.130085,-20.535725,-14.039542,-10.792073,-10.141679,-9.522157,...,-9.510473,-9.627995,-10.247801,-9.474483,-11.393511,-11.665322,-12.166050,-15.127309,-16.022055,fluent in english
191,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,-43.994282,...,-8.182457,-9.416603,-10.772792,-11.071528,-9.414461,-7.295956,-8.415949,-12.430440,-12.365926,Not fluent
289,-30.512718,-30.143410,-30.295103,-30.747799,-28.761522,-27.838215,-28.290575,-29.119884,-29.281870,-29.482321,...,-16.515652,-19.872427,-22.924105,-24.551083,-24.442820,-24.439196,-25.787098,-26.704235,-28.762295,Not fluent
12,-31.568993,-30.372770,-30.497868,-31.879025,-31.085445,-28.337587,-13.900833,-10.571099,-14.699902,-10.978611,...,-15.276581,-18.596012,-23.223011,-24.088108,-23.038807,-22.441883,-22.787474,-24.173750,-22.472755,fluent in english


In [20]:
rnewdf=rnewdf.fillna(0)

In [21]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

In [22]:
trainfeatures = train.iloc[:, :-1]

In [23]:
trainlabel = train.iloc[:, -1:]

In [24]:
testfeatures = test.iloc[:, :-1]

In [25]:
testlabel = test.iloc[:, -1:]

In [26]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [27]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [28]:
X_train.shape

(250, 1723)

In [29]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [30]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
# Edit according to target class no.
model.add(Dense(2))
model.add(Activation('softmax'))
opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)


In [31]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [32]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('model_accent.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models1/Accent_Detection_Model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 86.54%


In [33]:
preds = loaded_model.predict(x_testcnn, 
                         batch_size=32, 
                         verbose=1)

2/2 [==============================] - 7s 3s/step


In [34]:
preds

array([[0.5788431 , 0.42115685],
       [0.47879854, 0.5212015 ],
       [0.6545501 , 0.3454499 ],
       [0.59656394, 0.40343598],
       [0.41914332, 0.5808566 ],
       [0.19878867, 0.8012113 ],
       [0.5190581 , 0.48094192],
       [0.6559365 , 0.34406352],
       [0.6752203 , 0.32477972],
       [0.7443525 , 0.2556475 ],
       [0.7538809 , 0.24611911],
       [0.51887625, 0.4811237 ],
       [0.91821605, 0.08178391],
       [0.19153537, 0.80846465],
       [0.51523185, 0.4847682 ],
       [0.5507396 , 0.4492604 ],
       [0.69716424, 0.30283576],
       [0.4979389 , 0.50206107],
       [0.62562793, 0.37437207],
       [0.65733457, 0.34266537],
       [0.45427862, 0.5457214 ],
       [0.55302763, 0.44697243],
       [0.63473064, 0.36526933],
       [0.86587745, 0.13412255],
       [0.49162418, 0.5083758 ],
       [0.51230097, 0.48769903],
       [0.28363964, 0.7163604 ],
       [0.55866563, 0.44133437],
       [0.5969284 , 0.40307158],
       [0.7822271 , 0.21777292],
       [0.

In [35]:
preds1=preds.argmax(axis=1)

In [36]:
preds1

array([0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1], dtype=int64)

In [37]:
abc = preds1.astype(int).flatten()

In [38]:
predictions = (lb.inverse_transform((abc)))

In [39]:
preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:10]

,predictedvalues
0,Not fluent
1,fluent in english
2,Not fluent
3,Not fluent
4,fluent in english
5,fluent in english
6,Not fluent
7,Not fluent
8,Not fluent
9,Not fluent


In [40]:
actual=y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

In [41]:
actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:10]

,actualvalues
0,Not fluent
1,fluent in english
2,Not fluent
3,fluent in english
4,fluent in english
5,fluent in english
6,fluent in english
7,Not fluent
8,Not fluent
9,Not fluent


In [42]:
finaldf = actualdf.join(preddf)

In [43]:
finaldf[:5]

,actualvalues,predictedvalues
0,Not fluent,Not fluent
1,fluent in english,fluent in english
2,Not fluent,Not fluent
3,fluent in english,Not fluent
4,fluent in english,fluent in english


In [44]:
finaldf.groupby('actualvalues').count()

,predictedvalues
actualvalues,
Not fluent,30
fluent in english,22


In [45]:
finaldf.groupby('predictedvalues').count()

,actualvalues
predictedvalues,
Not fluent,37
fluent in english,15


## Testing Path!

In [46]:
% pylab inline
import os
import pandas as pd
import librosa
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

UsageError: Line magic function `%` not found.


In [66]:
#livedf= pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load('customer.wav', res_type='kaiser_fast',duration=20,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
featurelive = mfccs
livedf2 = featurelive

In [67]:
livedf2= pd.DataFrame(data=livedf2)

In [68]:
livedf2 = livedf2.stack().to_frame().T

In [69]:
twodim= np.expand_dims(livedf2, axis=2)

In [70]:
livepreds = loaded_model.predict(twodim, 
                         batch_size=32, 
                         verbose=1)

ValueError: in user code:

    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
  

In [62]:
livepreds

array([[0.55714995, 0.44285005]], dtype=float32)

In [63]:
livepreds1=livepreds.argmax(axis=1)

In [64]:
liveabc = livepreds1.astype(int).flatten()

In [65]:
livepredictions = (lb.inverse_transform((liveabc)))
livepredictions

array(['Not fluent'], dtype=object)